<a href="https://www.kaggle.com/code/cagkangrsy/rating-product-sorting-reviews-in-amazon?scriptVersionId=109837048" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Rating Product & Sorting Reviews in Amazon

## Business Problem
One of the most crucial problems in e-commerce is the correct calculation of the ratings of products after sale. The solution to this problem will provide more customer satisfaction. Also, the products of the sellers will be featured in the site and customers will have a problem free shopping experience. Another problem is that, the correct sorting of the comments. When misleading comments go up, it will influence the sales adversely which will result in both customer and financial losses. With the solution of these 2 fundamental problems the sales will rise and customers will have more smooth purchase experience.  


## Dataset Summary

- **reviewerID:** User ID  
- **asin:** Product ID  
- **reviewerName:** Username  
- **helpful:** Helpful evaluation rating  
- **reviewText:** Review  
- **overall:** Product rating  
- **summary:** Review summary  
- **unixReviewTime:** Date of review  
- **reviewTime:** Date of review Raw  
- **day_diff:** Number of days since review  
- **helpful_yes:** Number of votes the review has been found helpful  
- **total_vote:** Number of total votes on the review

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import math
import scipy.stats as st

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
df_ = pd.read_csv("../input/amazon-review/amazon_review.csv")
df = df_.copy()

In [3]:
# general_info function is written to observe the general shape of the dataframe.
def general_info (dataframe):
    print("------ Shape ------")
    print(dataframe.shape,"\n")
    print("------ Head ------")
    print(dataframe.head(10),"\n")
    print("------ Variables ------")
    print(dataframe.columns,"\n")
    print("------ Statistics ------")
    print(dataframe.describe().T,"\n")
    print("------ Null Value ------")
    print(dataframe.isnull().sum(),"\n")
    print("------ Variable Types ------")
    print(dataframe.info(),"\n")
general_info(df)

------ Shape ------
(4915, 12) 

------ Head ------
       reviewerID        asin                                      reviewerName helpful                                         reviewText  overall                                            summary  unixReviewTime  reviewTime  day_diff  helpful_yes  total_vote
0  A3SBTW3WS4IQSN  B007WTAJTO                                               NaN  [0, 0]                                         No issues.  4.00000                                         Four Stars      1406073600  2014-07-23       138            0           0
1  A18K1ODH1I2MVB  B007WTAJTO                                              0mie  [0, 0]  Purchased this for my device, it worked as adv...  5.00000                                      MOAR SPACE!!!      1382659200  2013-10-25       409            0           0
2  A2FII3I2MBMUIA  B007WTAJTO                                               1K3  [0, 0]  it works as expected. I should have sprung for...  4.00000               

In [4]:
# First the average of all ratings is observed. This can give a preliminary information about the product. 
df["overall"].mean()

4.587589013224822

In [5]:
# The reviewTime variable datatype is converted to date in order to make date calculations
df['reviewTime'] = pd.to_datetime(df['reviewTime'], dayfirst=True)\

# current_date variable is introduced as the latest day
current_date = df['reviewTime'].max()

# day_diff column is added to dataframe which is the difference of current date and the date review was made.
df["day_diff"] = (current_date - df['reviewTime']).dt.days

In [6]:

def time_weighted_average(dataframe, w1=32, w2=26, w3=24, w4=18):
    """
      Calculates quantile values of the dates and also calculates time based weighted average of time zones divided according to quantiles.

     parameters
     ----------
     dataframe: dataframe
     
     w1,w2,w3,w4: int
         Percent weights for time zones.

     Returns
     -------
     tw1 + tw2 + tw3 + tw4: float
         Time based weighted average
         
     averages_df: dataframe
         Presents weights and averages of the time zones

    """
    q1 = dataframe["day_diff"].quantile(0.25)
    q2 = dataframe["day_diff"].quantile(0.50)
    q3 = dataframe["day_diff"].quantile(0.75)

    tw1 = dataframe.loc[dataframe["day_diff"] <= q1, "overall"].mean() * w1 / 100
    tw2 = dataframe.loc[(dataframe["day_diff"] > q1) & (dataframe["day_diff"] <= q2), "overall"].mean() * w2 / 100 
    tw3 = dataframe.loc[(dataframe["day_diff"] > q2) & (dataframe["day_diff"] <= q3), "overall"].mean() * w3 / 100
    tw4 = dataframe.loc[(dataframe["day_diff"] > q3), "overall"].mean() * w4 / 100
    
    averages_df = pd.DataFrame({"weights": [w1,w2,w3,w4], "overall": [tw1/w1*100, tw2/w2*100, tw3/w3*100, tw4/w4*100]})
    
    return tw1 + tw2 + tw3 + tw4, averages_df 

time_based_weighted_average, averages_df = time_weighted_average(df)
print(f"Time based weighted average = {time_based_weighted_average}")
averages_df

Time based weighted average = 4.605574717472025


,weights,overall
0,32,4.69579
1,26,4.63614
2,24,4.57166
3,18,4.44625


In [7]:
# helpful_no variable is calculated, which is the count of comments that are not found helpful.
df["helpful_no"] = df["total_vote"] - df["helpful_yes"]

In [8]:
# df is trimmed to relevant variables for analysis.
df = df[["reviewerName", "overall", "summary", "helpful_yes", "helpful_no", "total_vote", "reviewTime"]] 

In [9]:
# The data is analyzed by positive and negative rating count difference by defining score_pos_neg_diff variable.
df["score_pos_neg_diff"] = df["helpful_yes"] - df["helpful_no"]
df.sort_values('score_pos_neg_diff', ascending=False).head(10)

# This analysis can fail in some situations. For example (2000 UP - 1500 DOWN) and (500 UP - 0 DOWN) has the same difference which will rank them same but there are 1500 unsatisfied customers on the first situation and there are none in the second. So, these two comments will influence readers differently.

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff
2031,"Hyoun Kim ""Faluzure""",5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1952,68,2020,2013-01-05,1884
4212,SkincareCEO,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1568,126,1694,2013-05-08,1442
3449,NLee the Engineer,5.00000,Top of the class among all (budget-priced) mic...,1428,77,1505,2012-09-26,1351
317,"Amazon Customer ""Kelly""",1.00000,"Warning, read this!",422,73,495,2012-02-09,349
3981,"R. Sutton, Jr. ""RWSynergy""",5.00000,"Resolving confusion between ""Mobile Ultra"" and...",112,27,139,2012-10-22,85
4596,"Tom Henriksen ""Doggy Diner""",1.00000,Designed incompatibility/Don't support SanDisk,82,27,109,2012-09-22,55
1835,goconfigure,5.00000,I own it,60,8,68,2014-02-28,52
4672,Twister,5.00000,Super high capacity!!! Excellent price (on Am...,45,4,49,2014-07-03,41
4306,Stellar Eller,5.00000,Awesome Card!,51,14,65,2012-09-06,37
315,"Amazon Customer ""johncrea""",5.00000,Samsung Galaxy Tab2 works with this card if re...,38,10,48,2012-08-13,28


In [10]:
# The data is analyzed by positive rating ratio by defining score_average_rating variable. 
df['score_average_rating'] = df.apply(lambda x : x["helpful_yes"] / x["total_vote"] if x["total_vote"] != 0 else 0,axis=1)
df.sort_values('score_average_rating', ascending=False).head(10)

# Likely, this analysis has a flaw too. If observed, it is seen that (500 UP - 0 DOWN) and (1 UP - O DOWN) has the same ratio of 1. These two comments would not influence the customer same. The "Wisdom of Crowds" effect will make customers to consider first situation more benefical. 

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff,score_average_rating
4277,S. Q.,5.00000,Perfect!!,1,0,1,2012-12-19,1,1.00000
2881,Lou Thomas,5.00000,Nexus One Loves This Card!,1,0,1,2012-01-10,1,1.00000
1073,C. Sanchez,5.00000,Tons of space for phone,1,0,1,2013-08-13,1,1.00000
445,"Apache ""Elizabeth""",4.00000,Amazon Great Prices,1,0,1,2013-12-18,1,1.00000
3923,Rock Your Roots,5.00000,What more to say?,1,0,1,2013-12-30,1,1.00000
435,Anthony L cate,5.00000,Love the extra storage,1,0,1,2012-07-24,1,1.00000
2901,luis,5.00000,Awesome and fast card :),1,0,1,2013-05-13,1,1.00000
2204,"jbwam ""jbwam""",2.00000,Sandisk will replace failures due to bad batch...,1,0,1,2013-06-14,1,1.00000
2206,JCBiker,5.00000,Great card,1,0,1,2013-10-31,1,1.00000
3408,"Neng Vang ""Neng2012""",5.00000,working no problem,1,0,1,2013-07-25,1,1.00000


In [11]:
def wilson_lower_bound(up, down, confidence=0.95):
    """
      Calculate Wilson Lower Bound Score

     - The lower limit of the confidence interval to be calculated for the Bernoulli parameter p is accepted as the WLB score.
     - The score to be calculated is used for product ranking.
     - Note:
     If the scores are between 1-5, 1-3 are marked as negative, 4-5 as positive and can be made to conform to Bernoulli.
     This brings with it some problems. For this reason, it is necessary to make a bayesian average rating.

     parameters
     ----------
     up: int
         Count of up ratings
     down: int
         Count of down ratings
     confidence: float
         Confidence interval

     Returns
     -------
     wilson score: float

    """
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

df['wilson_lower_bound'] = df.apply(lambda x: wilson_lower_bound(x["helpful_yes"],x["helpful_no"]),axis=1)
df.sort_values('wilson_lower_bound', ascending=False).head(10)

# In this analysis,it is observed that with 95% confidence the postive score ratio will be larger than the WLB score calculated. For example, randomly picking a value as "0.81858". This shows that with 95% confidence, this comment will have 81% positive ratings.

,reviewerName,overall,summary,helpful_yes,helpful_no,total_vote,reviewTime,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5.00000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1952,68,2020,2013-01-05,1884,0.96634,0.95754
3449,NLee the Engineer,5.00000,Top of the class among all (budget-priced) mic...,1428,77,1505,2012-09-26,1351,0.94884,0.93652
4212,SkincareCEO,1.00000,1 Star reviews - Micro SDXC card unmounts itse...,1568,126,1694,2013-05-08,1442,0.92562,0.91214
317,"Amazon Customer ""Kelly""",1.00000,"Warning, read this!",422,73,495,2012-02-09,349,0.85253,0.81858
4672,Twister,5.00000,Super high capacity!!! Excellent price (on Am...,45,4,49,2014-07-03,41,0.91837,0.80811
1835,goconfigure,5.00000,I own it,60,8,68,2014-02-28,52,0.88235,0.78465
3981,"R. Sutton, Jr. ""RWSynergy""",5.00000,"Resolving confusion between ""Mobile Ultra"" and...",112,27,139,2012-10-22,85,0.80576,0.73214
3807,R. Heisler,3.00000,"Good buy for the money but wait, I had an issue!",22,3,25,2013-02-27,19,0.88000,0.70044
4306,Stellar Eller,5.00000,Awesome Card!,51,14,65,2012-09-06,37,0.78462,0.67033
4596,"Tom Henriksen ""Doggy Diner""",1.00000,Designed incompatibility/Don't support SanDisk,82,27,109,2012-09-22,55,0.75229,0.66359
